In [1]:
import sys
sys.path.append('../..')
from lib import model_helper, dataset_generation, automated_evaluation, chat_helper
from anthropic import Anthropic
from lib.automated_evaluation import  caesar_decrypt
from transformers import AutoTokenizer
import pickle
import pandas as pd
from datagen_functions import affirming_answer, denying_answer,affirming_detail_answer,denying_detail_answer
from tqdm import tqdm
import torch
token = os.getenv('TOKEN_NAME')
import os
import re
import json


In [2]:
import openai
openai_api_key = openai_api_key = os.getenv('OPENAI_API_KEY')

In [3]:
question_path="../Datasets/HOCUS/questions/"
direct_questions = pd.read_csv(question_path+'direct_questions.csv')
alluding_questions = pd.read_csv(question_path+'alluding_questioning.csv')
questioning_assuming_statement= pd.read_csv(question_path+'questioning_assuming_statement.csv')
conversation = pd.read_csv(question_path+'conversation.csv')

In [4]:
i = 20
print("true statement:")
print(direct_questions["truth"][i])
print("false statement:")
print(direct_questions["fiction"][i])
print("direct question:")
print(direct_questions["fiction_question"][i])

print("questioning assuming statement:")
print(questioning_assuming_statement["fiction_question"][i])
print("alluding question:")
print(alluding_questions["fiction_question"][i])
print("conversation:")
print(conversation["fiction_question"][i])

true statement:
-  "Plants use chlorophyll to absorb sunlight in the process of photosynthesis."
false statement:
-  "Plants use chocolate to absorb sunlight in the process of photosynthesis."
direct question:
Do plants use chocolate to absorb sunlight in the process of photosynthesis?
questioning assuming statement:
Considering that plants use chocolate to absorb sunlight in the process of photosynthesis, how exactly does the chocolate facilitate the absorption of sunlight?
alluding question:
Recalling our previous conversation in which you mentioned that plants use chocolate to absorb sunlight during photosynthesis, could you please explain how exactly the chocolate facilitates the absorption of sunlight?
conversation:
Human: I've heard that plants use chocolate to absorb sunlight in the process of photosynthesis. Can you elaborate on that?
AI: Plants don't actually use chocolate to absorb sunlight in the process of photosynthesis.
Human: My apologies for the mistake. How exactly doe

In [5]:
print("Hallucinationg answers")
print(alluding_questions["fiction_accepetance"][i])
print("Not hallucinationg answers")
print(alluding_questions["fiction_rejection"][i])

Hallucinationg answers
Chocolate does not play a role in the process of photosynthesis for plants. The assumption made in the question is false.
Not hallucinationg answers
That assumption is incorrect. Plants do not use chocolate to absorb sunlight during photosynthesis.


In [6]:
def generate_answers(dataset,affirming_answer_funciton, denying_answer_funciton):
    truth_dict = {}
    truth_dict["questions"]= dataset["truth_question"].tolist()
    truth_dict["assumptions"] = dataset["truth"].tolist()
    truth_df = pd.DataFrame(truth_dict)
    fiction_dict = {}
    fiction_dict["questions"]= dataset["fiction_question"].tolist()
    fiction_dict["assumptions"] = dataset["fiction"].tolist()
    fiction_df = pd.DataFrame(fiction_dict)

    affirming_answer_funciton(truth_df)
    dataset["truth_accepetance"] = truth_df["question"]
    denying_answer_funciton(truth_df)
    dataset["truth_rejection"] = truth_df["question"]

    affirming_answer_funciton(fiction_df)
    dataset["fiction_accepetance"] = fiction_df["question"]
    denying_answer_funciton(fiction_df)
    dataset["fiction_rejection"] = fiction_df["question"]
    return dataset

In [6]:
direct_questions =generate_answers(direct_questions,affirming_answer, denying_answer)
direct_questions.to_csv(question_path+'direct_questions.csv',index=False)

In [7]:
alluding_questions_test = alluding_qu  estions[1000:1010].copy().reset_index(drop=True)
alluding_questions_test =generate_answers(alluding_questions_test,affirming_detail_answer, denying_detail_answer)

In [13]:
alluding_questions =generate_answers(alluding_questions,affirming_detail_answer, denying_detail_answer)
alluding_questions.to_csv(question_path+'alluding_questioning.csv',index=False)

In [ ]:
questioning_assuming_statement["truth_accepetance"] = alluding_questions["truth_accepetance"]
questioning_assuming_statement["truth_rejection"] = alluding_questions["truth_rejection"]
questioning_assuming_statement["fiction_accepetance"] = alluding_questions["fiction_accepetance"]
questioning_assuming_statement["fiction_rejection"] = alluding_questions["fiction_rejection"]
questioning_assuming_statement.to_csv(question_path+'questioning_assuming_statement.csv',index=False)


In [ ]:
conversation["truth_accepetance"] = alluding_questions["truth_accepetance"]
conversation["truth_rejection"] = alluding_questions["truth_rejection"]
conversation["fiction_accepetance"] = alluding_questions["fiction_accepetance"]
conversation["fiction_rejection"] = alluding_questions["fiction_rejection"]
conversation.to_csv(question_path+'conversation.csv',index=False)
